Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
display(data.head())
print(data.shape)
print("Дублей:", data.duplicated().sum())
print(data.info())
print(data.isnull().sum())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


(5000, 5)
Дублей: 153
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [4]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

### Вывод:

Я загрузила таблицу с данными клиентов страховой компании. Датафрейм состоит из 5000 строчек и 5 столбцов. Я поанализировала данные: пропусков нет, типы данных корректны, однако мной принято решении преобразовать данные в столбце "Возраст" и "Зарплата" в целочисленный тип данных int. Дублей 153, не удаляю их т.к., возможно это разные люди с одними и теми же признаками, и дубли не повлияют на сравнение показателей модели на "истинных" и зашифрованных данных.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится при умножении признаков на обратимую матрицу.

**Обоснование:** 

берем вместо матрицы X матрицы X*P и подставляем в формулу
$$
w1 = ((XР)^T XР)^{-1} (XР)^T y
$$

раскрываем скобки
$$
w1 = (X^T Р^T XР)^{-1} X^T Р^T y
$$

в -1 степени могут быть только квадратные матрицы.
учитывая св-во раскрываем скобки и сокращаем

$$
w1 = (X^T XР)^{-1} X^T y
$$

раскрываем скобки, финальное выражение
$$
w1 = (X^T X)^{-1} P^{-1} X^T y
$$

соотношение w к w1 равно P, т.е. $$ w/w1=P  =>  Pw1=w  =>  a = Xw = XPw1 $$
т.к. w и w1 это веса, то логично что при домножении признаков на обратную матрицу веса поменяются, однако это не повлияет на качество линейной регрессии.

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера</b>

Ответ верный!

</div>

<div class="alert alert-block alert-warning">
<b>Комментарий ревьюера</b>

Однако в обосновании есть небольшие неточности при открытии скобок. Умножаемые матрицы, как бы, меняются местами:
    
$$
(A \cdot B)^{-1} = B^{-1} \cdot A^{-1} \\
$$
    
$$
(A \cdot B)^T = B^T \cdot A^T \\
$$

</div>


## Алгоритм преобразования

**Алгоритм**

Алгоритм шифрования - умножение матрицы признаков на матрицу обратимую матрицу P размера 4 на 4 (по количеству столбцов (признаков) в матрице Х).

**Обоснование**

1)Соотношение w к w1 равно P, т.е. $$ w/w1=P  =>  Pw1=w  =>  a = Xw = XPw1 $$
т.к. w и w1 это веса, то логично что при домножении признаков на обратную матрицу веса поменяются, однако это не повлияет на качество линейной регрессии.


2)Если X.shape = (n, m), то P.shape = (m, m)

## Проверка алгоритма

In [5]:
features_matrix = data.drop(['Страховые выплаты'], axis=1).values
target_matrix = data['Страховые выплаты'].values

def check_model(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predicted = model.predict(features)
    print("R2 =", r2_score(target, predicted))

### Результаты при проверке на исходных данных

In [6]:
check_model(features_matrix, target_matrix)

R2 = 0.42494550308169177


### Результаты при проверке на зашифрованных данных

In [7]:
#проверка на обратимость
random_matrix = np.random.normal(size = (4,4))
try:
    np.linalg.inv(a)
except:
    random_matrix = np.random.normal(size = (4,4))

features_matrix = features_matrix @ random_matrix
check_model(features_matrix, target_matrix)

R2 = 0.42494550308170076


### Вывод:

Я проверила качество линейной регрессии на исходных и преобразованных данных. Можно утверждать, что качество линейной регрессии не отличается до и после преобразования. R2 на исходных равно 0.42494550308169177, на преобразованных данных: 0.42494550308170076. Таким образом, мы на практике подтвердили, чтобы защить данные клиентов и скрыть персональную информацию, мы  можем использовать наш метод преобразования данных, а именно домножение признаков на обратимую матрицу.